# VacationPy

---

## Import Libraries and Load Weather Coordinates Data

In [1]:
# Dependencies and libraries
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint
from api_keys import geoapify_key

In [2]:
# Read in saved cities.csv file and display as dataframe
city_data_df = pd.read_csv("../output_data/cities.csv")

city_data_df.head()

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tiksi,71.6872,128.8694,-23.85,99,100,2.72,RU,1707246038
1,1,talnakh,69.4865,88.3972,-19.89,88,100,3.08,RU,1707246040
2,2,sitka,57.0531,-135.3300,1.94,87,75,1.54,US,1707245939
3,3,grytviken,-54.2811,-36.5092,10.65,52,2,1.85,GS,1707246042
4,4,port-aux-francais,-49.3500,70.2167,6.15,86,100,12.23,TF,1707245856


---

### Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points("Lng", "Lat", geo=True, tiles="OSM", frame_width=900, frame_height=500,
                                      size="Humidity", scale=1, color="City", alpha=0.5)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
my_criteria_df = city_data_df.loc[(city_data_df["Max Temp"] <= 15) & (city_data_df["Max Temp"] >= -6)
                                  & (city_data_df["Cloudiness"] >= 70) & (city_data_df["Wind Speed"] <= 2)]

# Remove any cities with null values
my_criteria_df = my_criteria_df.dropna()

my_criteria_df

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,sitka,57.0531,-135.3300,1.94,87,75,1.54,US,1707245939
9,9,blackmans bay,-43.0167,147.3167,13.07,81,94,1.60,AU,1707246047
46,46,new norfolk,-42.7826,147.0587,13.72,77,100,1.08,AU,1707246073
47,47,kindberg,47.5000,15.4500,9.05,77,100,1.58,AT,1707246074
85,85,juneau,58.3019,-134.4197,0.23,100,100,1.39,US,1707246112
96,96,terrace,54.5163,-128.6035,-5.05,86,100,1.54,CA,1707246115
105,105,noshiro,40.2039,140.0242,-0.64,74,100,0.93,JP,1707246124
125,125,ketchikan,55.3422,-131.6461,1.99,77,100,1.27,US,1707246151
171,171,prince george,53.9166,-122.7530,-2.08,85,100,1.54,CA,1707246088
179,179,homer,59.6425,-151.5483,-4.06,100,100,0.00,US,1707246185


### Create a new DataFrame called `hotel_df`.

In [5]:
# Create new dataframe with city, lat, lng, max temp, humidity columns
hotel_df = my_criteria_df.copy()[["City", "Lat", "Lng", "Max Temp", "Humidity", "Country"]]

# Add a blank column titled Hotel Name
hotel_df["Hotel Name"] = ""

hotel_df

,City,Lat,Lng,Max Temp,Humidity,Country,Hotel Name
2,sitka,57.0531,-135.3300,1.94,87,US,
9,blackmans bay,-43.0167,147.3167,13.07,81,AU,
46,new norfolk,-42.7826,147.0587,13.72,77,AU,
47,kindberg,47.5000,15.4500,9.05,77,AT,
85,juneau,58.3019,-134.4197,0.23,100,US,
96,terrace,54.5163,-128.6035,-5.05,86,CA,
105,noshiro,40.2039,140.0242,-0.64,74,JP,
125,ketchikan,55.3422,-131.6461,1.99,77,US,
171,prince george,53.9166,-122.7530,-2.08,85,CA,
179,homer,59.6425,-151.5483,-4.06,100,US,


### For each city, use the Geoapify API to find the first hotel located within 10,000 meters of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories":"accommodation.hotel",
          "limit":5,
          "apiKey":geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
sitka - nearest hotel: Aspen Suites Hotel
blackmans bay - nearest hotel: Villa Howden
new norfolk - nearest hotel: Woodbridge on the Derwent
kindberg - nearest hotel: Hölzl
juneau - nearest hotel: Ramada by Wyndham Juneau
terrace - nearest hotel: Sandman Hotel Terrace
noshiro - nearest hotel: あづま旅館
ketchikan - nearest hotel: Cape Fox Lodge
prince george - nearest hotel: Travelodge by Wyndham Prince George
homer - nearest hotel: Driftwood Inn & Homer Seaside Lodges
portland - nearest hotel: the Hoxton
lanxi - nearest hotel: No hotel found
levuka - nearest hotel: No hotel found
nabeul - nearest hotel: Dar Oum El Khair
nakamura - nearest hotel: ホテル アイランド
tenno - nearest hotel: 男鹿の屋
bryne - nearest hotel: Jæren hotell
bomdila - nearest hotel: Seagull
mammoth lakes - nearest hotel: Travelodge by Wyndham Mammoth Lakes
dryden - nearest hotel: Holiday Inn Express
iwanai - nearest hotel: ホチルいのう
pokhara - nearest hotel: Hotel Annapurna View Sarangkot
xiongzhou - nearest hot

,City,Lat,Lng,Max Temp,Humidity,Country,Hotel Name
2,sitka,57.0531,-135.3300,1.94,87,US,Aspen Suites Hotel
9,blackmans bay,-43.0167,147.3167,13.07,81,AU,Villa Howden
46,new norfolk,-42.7826,147.0587,13.72,77,AU,Woodbridge on the Derwent
47,kindberg,47.5000,15.4500,9.05,77,AT,Hölzl
85,juneau,58.3019,-134.4197,0.23,100,US,Ramada by Wyndham Juneau
96,terrace,54.5163,-128.6035,-5.05,86,CA,Sandman Hotel Terrace
105,noshiro,40.2039,140.0242,-0.64,74,JP,あづま旅館
125,ketchikan,55.3422,-131.6461,1.99,77,US,Cape Fox Lodge
171,prince george,53.9166,-122.7530,-2.08,85,CA,Travelodge by Wyndham Prince George
179,homer,59.6425,-151.5483,-4.06,100,US,Driftwood Inn & Homer Seaside Lodges


### Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points("Lng", "Lat", geo=True, tiles="OSM", frame_width=900, frame_height=500,
                                      size="Humidity", scale=1, color="City", alpha=0.5, hover_cols=["Hotel Name", "Country"])

# Display the map
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)